In [3]:
import pandas as pd
import numpy as np
import os

df= pd.read_csv('skill2vec_10k.csv')
df.head()

C:\Users\pveer\AppData\Local\Temp\ipykernel_13684\3684493577.py:5: DtypeWarning: Columns (34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,

,4374,bapi,sap_badi,sap_abap,sap_alv,user_exit,dialog_programming,sapscript,ale,floorplan_manager,...,Unnamed: 951,Unnamed: 952,Unnamed: 953,Unnamed: 954,Unnamed: 955,Unnamed: 956,Unnamed: 957,Unnamed: 958,Unnamed: 959,Unnamed: 960
0,101457,Data Loss Prevention,Networking,Firewall,Security,Network Administration,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,108811,Litigation Management,Litigation,contracts,Legal,drafting,vetting,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,95500,SFDC,Salesforce,Salesforce developer,Java,J2ee,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64897,Intrusion Detection,Networking,LAN,Linux,Unix,Firewall,WAN,Technical Support,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20795,Business Development Management,Concept Selling,Hotel Management,Target Achievement,horeca,caterers,Sales,Bakery,B2B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(9999, 961)

In [5]:
# Check column names and data types
print("Number of columns:", len(df.columns))
print("First 20 column names:")
print(df.columns[:20].tolist())
print("\nLast 20 column names:")
print(df.columns[-20:].tolist())
print("\nData types:")
print(df.dtypes.value_counts())

Number of columns: 961
First 20 column names:
['4374', 'bapi', 'sap_badi', 'sap_abap', 'sap_alv', 'user_exit', 'dialog_programming', 'sapscript', 'ale', 'floorplan_manager', 'sap_pi', 'brf', 'sap_netweaver', 'sap_edi', 'bdc', 'bcs', 'fast_search', 'k2_blackpearl', 'workflow', 'catt']

Last 20 column names:
['Unnamed: 941', 'Unnamed: 942', 'Unnamed: 943', 'Unnamed: 944', 'Unnamed: 945', 'Unnamed: 946', 'Unnamed: 947', 'Unnamed: 948', 'Unnamed: 949', 'Unnamed: 950', 'Unnamed: 951', 'Unnamed: 952', 'Unnamed: 953', 'Unnamed: 954', 'Unnamed: 955', 'Unnamed: 956', 'Unnamed: 957', 'Unnamed: 958', 'Unnamed: 959', 'Unnamed: 960']

Data types:
object    960
int64       1
Name: count, dtype: int64


In [6]:
# Strategy to merge 961 columns into 10 columns
# We'll group the columns and combine them in a meaningful way

# First, let's identify the structure
print("Checking for any non-null values in unnamed columns...")
unnamed_cols = [col for col in df.columns if 'Unnamed' in str(col)]
print(f"Number of unnamed columns: {len(unnamed_cols)}")

# Check if unnamed columns have any data
unnamed_data_check = df[unnamed_cols].notna().sum().sum()
print(f"Total non-null values in unnamed columns: {unnamed_data_check}")

# Get the meaningful columns (excluding unnamed ones)
meaningful_cols = [col for col in df.columns if 'Unnamed' not in str(col)]
print(f"Number of meaningful columns: {len(meaningful_cols)}")
print("First few meaningful columns:", meaningful_cols[:10])

Checking for any non-null values in unnamed columns...
Number of unnamed columns: 929
Total non-null values in unnamed columns: 2807
Number of meaningful columns: 32
First few meaningful columns: ['4374', 'bapi', 'sap_badi', 'sap_abap', 'sap_alv', 'user_exit', 'dialog_programming', 'sapscript', 'ale', 'floorplan_manager']
Total non-null values in unnamed columns: 2807
Number of meaningful columns: 32
First few meaningful columns: ['4374', 'bapi', 'sap_badi', 'sap_abap', 'sap_alv', 'user_exit', 'dialog_programming', 'sapscript', 'ale', 'floorplan_manager']


In [7]:
# Create a new DataFrame with 10 merged columns
# Let's categorize the skills into logical groups

# First, let's examine what kind of data we have in meaningful columns
print("Sample of meaningful columns data:")
sample_cols = meaningful_cols[:5]
for col in sample_cols:
    print(f"\n{col}:")
    print(df[col].value_counts().head(3))

Sample of meaningful columns data:

4374:
4374
101457    1
110874    1
13434     1
Name: count, dtype: int64

bapi:
bapi
development             171
HR                       88
Communication Skills     82
Name: count, dtype: int64

sap_badi:
sap_badi
Java                    66
Communication Skills    63
development             39
Name: count, dtype: int64

sap_abap:
sap_abap
Communication Skills    64
Java                    58
Sales                   47
Name: count, dtype: int64

sap_alv:
sap_alv
Sales         57
Java          56
Javascript    51
Name: count, dtype: int64


In [8]:
# Define the 10 categories for merging columns
# We'll group columns based on their names and content patterns

# Create category mappings
category_mapping = {
    'ID_Info': ['4374'],  # Keeping the ID column separate
    'SAP_Technologies': [],
    'Programming_Languages': [],
    'Databases': [],
    'Web_Technologies': [],
    'Communication_Skills': [],
    'Business_Skills': [],
    'Technical_Skills': [],
    'Development_Tools': [],
    'Other_Skills': []
}

# Categorize meaningful columns based on their names
for col in meaningful_cols[1:]:  # Skip the first column (ID)
    col_lower = col.lower()
    if 'sap' in col_lower or 'bapi' in col_lower or 'badi' in col_lower or 'abap' in col_lower:
        category_mapping['SAP_Technologies'].append(col)
    elif any(prog in col_lower for prog in ['java', 'python', 'javascript', 'c++', 'programming']):
        category_mapping['Programming_Languages'].append(col)
    elif any(db in col_lower for db in ['sql', 'database', 'oracle', 'mysql', 'mongodb']):
        category_mapping['Databases'].append(col)
    elif any(web in col_lower for web in ['web', 'html', 'css', 'http', 'api', 'rest']):
        category_mapping['Web_Technologies'].append(col)
    elif any(comm in col_lower for comm in ['communication', 'presentation', 'team']):
        category_mapping['Communication_Skills'].append(col)
    elif any(biz in col_lower for biz in ['business', 'management', 'sales', 'marketing']):
        category_mapping['Business_Skills'].append(col)
    elif any(dev in col_lower for dev in ['development', 'testing', 'debug', 'deployment']):
        category_mapping['Development_Tools'].append(col)
    elif any(tech in col_lower for tech in ['technical', 'system', 'network', 'security']):
        category_mapping['Technical_Skills'].append(col)
    else:
        category_mapping['Other_Skills'].append(col)

# Print the categorization
for category, columns in category_mapping.items():
    print(f"{category}: {len(columns)} columns")
    if columns:
        print(f"  Examples: {columns[:3]}")
    print()

ID_Info: 1 columns
  Examples: ['4374']

SAP_Technologies: 13 columns
  Examples: ['bapi', 'sap_badi', 'sap_abap']

Programming_Languages: 1 columns
  Examples: ['dialog_programming']

Databases: 0 columns

Web_Technologies: 0 columns

Communication_Skills: 0 columns

Business_Skills: 1 columns
  Examples: ['invoice_management']

Technical_Skills: 0 columns

Development_Tools: 0 columns

Other_Skills: 16 columns
  Examples: ['user_exit', 'ale', 'floorplan_manager']



In [9]:
# Create the merged DataFrame with 10 columns
merged_df = pd.DataFrame()

# Function to combine values from multiple columns
def combine_columns(df, columns):
    """Combine multiple columns into a single column with meaningful aggregation"""
    if not columns:
        return pd.Series([''] * len(df), index=df.index)
    
    # For each row, combine non-null values from all specified columns
    combined_series = pd.Series([''] * len(df), index=df.index)
    
    for idx in df.index:
        values = []
        for col in columns:
            if col in df.columns:
                val = df.loc[idx, col]
                if pd.notna(val) and str(val).strip() != '':
                    values.append(str(val))
        
        # Join unique values with semicolon
        if values:
            unique_values = list(dict.fromkeys(values))  # Remove duplicates while preserving order
            combined_series.loc[idx] = '; '.join(unique_values)
    
    return combined_series

# Create the 10 merged columns
for category, columns in category_mapping.items():
    if category == 'ID_Info':
        # Keep ID as is
        merged_df[category] = df[columns[0]] if columns else ''
    else:
        # Combine all columns in this category
        merged_df[category] = combine_columns(df, columns)

print("Merged DataFrame shape:", merged_df.shape)
print("\nColumn names:")
print(merged_df.columns.tolist())

Merged DataFrame shape: (9999, 10)

Column names:
['ID_Info', 'SAP_Technologies', 'Programming_Languages', 'Databases', 'Web_Technologies', 'Communication_Skills', 'Business_Skills', 'Technical_Skills', 'Development_Tools', 'Other_Skills']


In [10]:
# Now let's also merge the unnamed columns into our "Other_Skills" category
# since they might contain additional skill information

print("Adding unnamed columns to Other_Skills...")

# Get non-empty values from unnamed columns and add to Other_Skills
for idx in df.index:
    unnamed_values = []
    for col in unnamed_cols:
        val = df.loc[idx, col]
        if pd.notna(val) and str(val).strip() != '':
            unnamed_values.append(str(val))
    
    if unnamed_values:
        # Combine with existing Other_Skills values
        existing_other = merged_df.loc[idx, 'Other_Skills']
        if existing_other:
            all_values = existing_other.split('; ') + unnamed_values
        else:
            all_values = unnamed_values
        
        # Remove duplicates and update
        unique_values = list(dict.fromkeys(all_values))
        merged_df.loc[idx, 'Other_Skills'] = '; '.join(unique_values)

print("Updated merged DataFrame shape:", merged_df.shape)

# Display sample results
print("\nSample of merged data:")
print(merged_df.head())

Adding unnamed columns to Other_Skills...
Updated merged DataFrame shape: (9999, 10)

Sample of merged data:
   ID_Info                                   SAP_Technologies  \
0   101457  Data Loss Prevention; Networking; Firewall; Se...   
1   108811  Litigation Management; Litigation; contracts; ...   
2    95500       SFDC; Salesforce; Salesforce developer; Java   
3    64897   Intrusion Detection; Networking; LAN; Linux; WAN   
4    20795  Business Development Management; Concept Selli...   

  Programming_Languages Databases Web_Technologies Communication_Skills  \
0                                                                         
1               vetting                                                   
2                                                                         
3              Firewall                                                   
4              caterers                                                   

  Business_Skills Technical_Skills Development_To

In [11]:
# Summary of the merged DataFrame
print("=== COLUMN MERGING SUMMARY ===")
print(f"Original DataFrame: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"Merged DataFrame: {merged_df.shape[0]} rows × {merged_df.shape[1]} columns")
print(f"Reduction: {df.shape[1] - merged_df.shape[1]} columns eliminated")

print("\n=== 10 MERGED COLUMNS ===")
for i, col in enumerate(merged_df.columns, 1):
    non_empty = (merged_df[col] != '').sum()
    print(f"{i:2d}. {col:<25} - {non_empty:,} records with data")

print("\n=== SAMPLE DATA FOR EACH COLUMN ===")
for col in merged_df.columns:
    print(f"\n{col}:")
    sample_data = merged_df[merged_df[col] != ''][col].head(3)
    if len(sample_data) > 0:
        for idx, value in sample_data.items():
            # Truncate long values for display
            display_value = value[:100] + "..." if len(str(value)) > 100 else value
            print(f"  Row {idx}: {display_value}")
    else:
        print("  No data available")

print("\n=== EXPORT OPTIONS ===")
print("To save the merged DataFrame:")
print("merged_df.to_csv('merged_skill_data_10_columns.csv', index=False)")
print("merged_df.to_excel('merged_skill_data_10_columns.xlsx', index=False)")

=== COLUMN MERGING SUMMARY ===
Original DataFrame: 9999 rows × 961 columns
Merged DataFrame: 9999 rows × 10 columns
Reduction: 951 columns eliminated

=== 10 MERGED COLUMNS ===
 1. ID_Info                   - 9,999 records with data
 2. SAP_Technologies          - 9,999 records with data
 3. Programming_Languages     - 6,983 records with data
 4. Databases                 - 0 records with data
 5. Web_Technologies          - 0 records with data
 6. Communication_Skills      - 0 records with data
 7. Business_Skills           - 464 records with data
 8. Technical_Skills          - 0 records with data
 9. Development_Tools         - 0 records with data
10. Other_Skills              - 7,716 records with data

=== SAMPLE DATA FOR EACH COLUMN ===

ID_Info:
  Row 0: 101457
  Row 1: 108811
  Row 2: 95500

SAP_Technologies:
  Row 0: Data Loss Prevention; Networking; Firewall; Security
  Row 1: Litigation Management; Litigation; contracts; Legal
  Row 2: SFDC; Salesforce; Salesforce developer; 

In [12]:
# Save the merged DataFrame
output_filename = 'merged_skill_data_10_columns.csv'
merged_df.to_csv(output_filename, index=False)

print(f"✅ Successfully saved merged data to: {output_filename}")
print(f"📊 Final dataset: {merged_df.shape[0]:,} rows × {merged_df.shape[1]} columns")
print(f"🔄 Reduced from {df.shape[1]} to {merged_df.shape[1]} columns ({((df.shape[1] - merged_df.shape[1]) / df.shape[1] * 100):.1f}% reduction)")

# Display the final merged DataFrame
print("\n📋 Final merged DataFrame:")
merged_df.head(10)

✅ Successfully saved merged data to: merged_skill_data_10_columns.csv
📊 Final dataset: 9,999 rows × 10 columns
🔄 Reduced from 961 to 10 columns (99.0% reduction)

📋 Final merged DataFrame:


,ID_Info,SAP_Technologies,Programming_Languages,Databases,Web_Technologies,Communication_Skills,Business_Skills,Technical_Skills,Development_Tools,Other_Skills
0,101457,Data Loss Prevention; Networking; Firewall; Se...,,,,,,,,Network Administration
1,108811,Litigation Management; Litigation; contracts; ...,vetting,,,,,,,drafting
2,95500,SFDC; Salesforce; Salesforce developer; Java,,,,,,,,J2ee
3,64897,Intrusion Detection; Networking; LAN; Linux; WAN,Firewall,,,,,,,Unix; Technical Support
4,20795,Business Development Management; Concept Selli...,caterers,,,,,,,horeca; Bakery; B2B
5,23036,business analysis; trade lifecycle; business a...,derivatives,,,,,,,otc
6,55819,Outbound Calls; cold calls; inbound calls; voi...,CRM,,,,,,,Customer Care
7,34339,CISA; DISA; CISSP; CEH; Certified Information ...,information Technology,,,,,,,CIA; internal audit; audit; Technology audit; ...
8,48408,KPO; HR Assistance; Communication Skills; HR; ...,Operations,,,,,,,hr generalist; onboarding
9,107414,WPF; WCF; C#; Java; SQL; Web Technologies,Web Services,,,,,,,.NET Framework; Oracle PL; Software Development


In [13]:
merged_df['SAP_Technologies']

0       Data Loss Prevention; Networking; Firewall; Se...
1       Litigation Management; Litigation; contracts; ...
2            SFDC; Salesforce; Salesforce developer; Java
3        Intrusion Detection; Networking; LAN; Linux; WAN
4       Business Development Management; Concept Selli...
                              ...                        
9994    Providing Training; Capacity Building; trainer...
9995        Salesforce.com; sfdc; crm; salesforce dot com
9996    Sales Management; Business Development; Lead g...
9997    inbound; Insurance Sales; insurance marketing;...
9998    Business Finance; Financial Analysis; Cfo; Fin...
Name: SAP_Technologies, Length: 9999, dtype: object

In [14]:
merged_df['SAP_Technologies'][1]

'Litigation Management; Litigation; contracts; Legal'

In [15]:
# Extract all unique SAP skills from the merged data
sap_skills_array = merged_df['SAP_Technologies'].dropna().str.split('; ').explode().dropna().unique()
print(f"Total unique SAP skills: {len(sap_skills_array)}")
print("\nFirst 20 unique SAP skills:")
print(sap_skills_array[:20].tolist())

Total unique SAP skills: 20260

First 20 unique SAP skills:
['Data Loss Prevention', 'Networking', 'Firewall', 'Security', 'Litigation Management', 'Litigation', 'contracts', 'Legal', 'SFDC', 'Salesforce', 'Salesforce developer', 'Java', 'Intrusion Detection', 'LAN', 'Linux', 'WAN', 'Business Development Management', 'Concept Selling', 'Hotel Management', 'Target Achievement']


In [16]:
sap_skills_array = merged_df['SAP_Technologies'].dropna().str.split('; ').explode().dropna().unique()

In [17]:
len(sap_skills_array)

20260

In [18]:
import json
from datetime import datetime

# Save the unique SAP skills to a JSON file with proper structure
sap_skills_json = 'sap_skills.json'
sap_skills_data = {
    "skills": sap_skills_array.tolist(),
    "total_count": len(sap_skills_array),
    "generated_on": datetime.now().isoformat(),
    "source": "merged_skill_data_10_columns.csv"
}

with open(sap_skills_json, 'w') as f:
    json.dump(sap_skills_data, f, indent=2)
print(f"✅ Successfully saved unique SAP skills to: {sap_skills_json}")
print(f"Total unique SAP skills saved: {len(sap_skills_array)}")

✅ Successfully saved unique SAP skills to: sap_skills.json
Total unique SAP skills saved: 20260


In [19]:
Programming_Languages_df= merged_df['Programming_Languages']
print(f"Total unique programming languages: {Programming_Languages_df.nunique()}")
print("\nFirst 20 unique programming languages:")   
Programming_Languages_df.head(20)

Total unique programming languages: 3997

First 20 unique programming languages:


0                            
1                     vetting
2                            
3                    Firewall
4                    caterers
5                 derivatives
6                         CRM
7      information Technology
8                  Operations
9                Web Services
10                           
11                           
12                           
13            ca intermediate
14                online work
15         Project Management
16                           
17                           
18    Relationship Management
19        Wage Administration
Name: Programming_Languages, dtype: object

In [20]:
Programming_Languages_df = merged_df['Programming_Languages'].dropna().explode().dropna().unique()
print(f"Total unique programming languages: {len(Programming_Languages_df)}")
print("\nFirst 20 unique programming languages:")
print(Programming_Languages_df[:20].tolist())

Total unique programming languages: 3997

First 20 unique programming languages:
['', 'vetting', 'Firewall', 'caterers', 'derivatives', 'CRM', 'information Technology', 'Operations', 'Web Services', 'ca intermediate', 'online work', 'Project Management', 'Relationship Management', 'Wage Administration', 'operational excellence', 'Export Marketing', 'drupal', 'ExtJS', 'advertising', 'Training Delivery']


In [21]:
merged_df.columns

Index(['ID_Info', 'SAP_Technologies', 'Programming_Languages', 'Databases',
       'Web_Technologies', 'Communication_Skills', 'Business_Skills',
       'Technical_Skills', 'Development_Tools', 'Other_Skills'],
      dtype='object')

In [22]:
SAP_Technologies_df=merged_df['SAP_Technologies'].dropna().str.split('; ').explode().dropna().unique()
print(f"Total unique SAP technologies: {len(SAP_Technologies_df)}")
print("\nFirst 20 unique SAP technologies:")
SAP_Technologies_df[:20].tolist()

Total unique SAP technologies: 20260

First 20 unique SAP technologies:


['Data Loss Prevention',
 'Networking',
 'Firewall',
 'Security',
 'Litigation Management',
 'Litigation',
 'contracts',
 'Legal',
 'SFDC',
 'Salesforce',
 'Salesforce developer',
 'Java',
 'Intrusion Detection',
 'LAN',
 'Linux',
 'WAN',
 'Business Development Management',
 'Concept Selling',
 'Hotel Management',
 'Target Achievement']

In [23]:
Programming_Languages_df= merged_df['Programming_Languages'].dropna().explode().dropna().unique()
print(f"Total unique programming languages: {len(Programming_Languages_df)}")
print("\nFirst 20 unique programming languages:")
print(Programming_Languages_df[:20].tolist())


Total unique programming languages: 3997

First 20 unique programming languages:
['', 'vetting', 'Firewall', 'caterers', 'derivatives', 'CRM', 'information Technology', 'Operations', 'Web Services', 'ca intermediate', 'online work', 'Project Management', 'Relationship Management', 'Wage Administration', 'operational excellence', 'Export Marketing', 'drupal', 'ExtJS', 'advertising', 'Training Delivery']


In [24]:
Databases_df = merged_df['Databases']
print(Databases_df.head())
print(f"Total unique databases: {Databases_df.nunique()}")
Databases_df = merged_df['Databases'].dropna().explode().dropna().unique()
print(f"Total unique databases: {len(Databases_df)}")
print("\nFirst 20 unique databases:")
print(Databases_df[:20].tolist())

0    
1    
2    
3    
4    
Name: Databases, dtype: object
Total unique databases: 1
Total unique databases: 1

First 20 unique databases:
['']


In [25]:
Web_Technologies_df = merged_df['Web_Technologies']
print(Web_Technologies_df.head())
print(f"Total unique web technologies: {Web_Technologies_df.nunique()}")
print("total nans in Web_Technologies column:", Web_Technologies_df.isna().sum())

0    
1    
2    
3    
4    
Name: Web_Technologies, dtype: object
Total unique web technologies: 1
total nans in Web_Technologies column: 0


In [26]:
Other_Skills_df = merged_df['Other_Skills']
print(Other_Skills_df.head())
print(f"Total unique other skills: {Other_Skills_df.nunique()}")
print("total nans in Other_Skills column:", Other_Skills_df.isna().sum())

0     Network Administration
1                   drafting
2                       J2ee
3    Unix; Technical Support
4        horeca; Bakery; B2B
Name: Other_Skills, dtype: object
Total unique other skills: 7041
total nans in Other_Skills column: 0


#### final json file fixed with the sap ,programing and other skills

In [27]:
SAP_Technologies_df=merged_df['SAP_Technologies'].dropna().str.split('; ').explode().dropna().unique()
print(f"Total unique SAP technologies: {len(SAP_Technologies_df)}")
print("\nFirst 20 unique SAP technologies:")
SAP_Technologies_df[:20].tolist()

Total unique SAP technologies: 20260

First 20 unique SAP technologies:


First 20 unique SAP technologies:


['Data Loss Prevention',
 'Networking',
 'Firewall',
 'Security',
 'Litigation Management',
 'Litigation',
 'contracts',
 'Legal',
 'SFDC',
 'Salesforce',
 'Salesforce developer',
 'Java',
 'Intrusion Detection',
 'LAN',
 'Linux',
 'WAN',
 'Business Development Management',
 'Concept Selling',
 'Hotel Management',
 'Target Achievement']

In [28]:
Programming_Languages_df= merged_df['Programming_Languages'].dropna().explode().dropna().unique()
print(f"Total unique programming languages: {len(Programming_Languages_df)}")
print("\nFirst 20 unique programming languages:")
print(Programming_Languages_df[:20].tolist())


Total unique programming languages: 3997

First 20 unique programming languages:
['', 'vetting', 'Firewall', 'caterers', 'derivatives', 'CRM', 'information Technology', 'Operations', 'Web Services', 'ca intermediate', 'online work', 'Project Management', 'Relationship Management', 'Wage Administration', 'operational excellence', 'Export Marketing', 'drupal', 'ExtJS', 'advertising', 'Training Delivery']


In [29]:
Other_Skills_df=    merged_df['Other_Skills'].dropna().str.split('; ').explode().dropna().unique()
print(f"Total unique other skills: {len(Other_Skills_df)}")
print("\nFirst 20 unique other skills:")
print(Other_Skills_df[:20].tolist())

Total unique other skills: 13513

First 20 unique other skills:
['Network Administration', 'drafting', 'J2ee', 'Unix', 'Technical Support', 'horeca', 'Bakery', 'B2B', 'otc', 'Customer Care', 'CIA', 'internal audit', 'audit', 'Technology audit', 'information security audit', 'information system Audit', 'hr generalist', 'onboarding', '.NET Framework', 'Oracle PL']


In [30]:
#conver the  sap ,programing and other skills to json file in single json file like {'skills':[]}
import json
sap_skills_json = 'skills.json'
skills_data = {
    "skills": {
        "SAP_Technologies": SAP_Technologies_df.tolist(),
        "Programming_Languages": Programming_Languages_df.tolist(),
        "Other_Skills": Other_Skills_df.tolist()
    },
    "total_count": {
        "SAP_Technologies": len(SAP_Technologies_df),
        "Programming_Languages": len(Programming_Languages_df),
        "Databases": len(Databases_df),
        "Web_Technologies": len(Web_Technologies_df),
        "Other_Skills": len(Other_Skills_df)
    },
    "generated_on": datetime.now().isoformat(),
    "source": "merged_skill_data_10_columns.csv"
}
with open(sap_skills_json, 'w') as f:
    json.dump(skills_data, f, indent=2)
print(f"✅ Successfully saved skills data to: {sap_skills_json}")
print(f"Total unique skills saved: {skills_data['total_count']}")


✅ Successfully saved skills data to: skills.json
Total unique skills saved: {'SAP_Technologies': 20260, 'Programming_Languages': 3997, 'Databases': 1, 'Web_Technologies': 9999, 'Other_Skills': 13513}


In [31]:
import json
from datetime import datetime

# Extract all unique skills from each category in merged_df
print("Extracting skills from merged DataFrame...")

# Extract SAP Technologies
sap_skills = merged_df['SAP_Technologies'].dropna().str.split('; ').explode().dropna().unique()
print(f"SAP Technologies: {len(sap_skills)} unique skills")

# Extract Programming Languages
prog_skills = merged_df['Programming_Languages'].dropna().str.split('; ').explode().dropna().unique()
print(f"Programming Languages: {len(prog_skills)} unique skills")

# Extract Databases
db_skills = merged_df['Databases'].dropna().str.split('; ').explode().dropna().unique()
print(f"Databases: {len(db_skills)} unique skills")

# Extract Web Technologies
web_skills = merged_df['Web_Technologies'].dropna().str.split('; ').explode().dropna().unique()
print(f"Web Technologies: {len(web_skills)} unique skills")

# Extract Business Skills
business_skills = merged_df['Business_Skills'].dropna().str.split('; ').explode().dropna().unique()
print(f"Business Skills: {len(business_skills)} unique skills")

# Extract Technical Skills
tech_skills = merged_df['Technical_Skills'].dropna().str.split('; ').explode().dropna().unique()
print(f"Technical Skills: {len(tech_skills)} unique skills")

# Extract Development Tools
dev_skills = merged_df['Development_Tools'].dropna().str.split('; ').explode().dropna().unique()
print(f"Development Tools: {len(dev_skills)} unique skills")

# Extract Communication Skills
comm_skills = merged_df['Communication_Skills'].dropna().str.split('; ').explode().dropna().unique()
print(f"Communication Skills: {len(comm_skills)} unique skills")

# Extract Other Skills
other_skills = merged_df['Other_Skills'].dropna().str.split('; ').explode().dropna().unique()
print(f"Other Skills: {len(other_skills)} unique skills")

# Combine all skills into a single array
all_skills = []
all_skills.extend(sap_skills.tolist())
all_skills.extend(prog_skills.tolist())
all_skills.extend(db_skills.tolist())
all_skills.extend(web_skills.tolist())
all_skills.extend(business_skills.tolist())
all_skills.extend(tech_skills.tolist())
all_skills.extend(dev_skills.tolist())
all_skills.extend(comm_skills.tolist())
all_skills.extend(other_skills.tolist())

# Remove duplicates while preserving order
unique_all_skills = list(dict.fromkeys(all_skills))

print(f"\n📊 Total skills before deduplication: {len(all_skills):,}")
print(f"📊 Total unique skills after deduplication: {len(unique_all_skills):,}")

# Create the final JSON structure with all skills in a single array
combined_skills_data = {
    "skills": unique_all_skills,
    "total_count": len(unique_all_skills),
    "breakdown": {
        "SAP_Technologies": len(sap_skills),
        "Programming_Languages": len(prog_skills),
        "Databases": len(db_skills),
        "Web_Technologies": len(web_skills),
        "Business_Skills": len(business_skills),
        "Technical_Skills": len(tech_skills),
        "Development_Tools": len(dev_skills),
        "Communication_Skills": len(comm_skills),
        "Other_Skills": len(other_skills)
    },
    "generated_on": datetime.now().isoformat(),
    "source": "merged_skill_data_10_columns.csv"
}

# Save to JSON file
skills_json_file = 'all_skills_combined.json'
with open(skills_json_file, 'w', encoding='utf-8') as f:
    json.dump(combined_skills_data, f, indent=2, ensure_ascii=False)

print(f"\n✅ Successfully saved all skills to: {skills_json_file}")
print(f"📋 Breakdown by category:")
for category, count in combined_skills_data['breakdown'].items():
    print(f"   - {category}: {count:,} skills")

# Display first 10 skills as sample
print(f"\n📝 First 10 skills:")
for i, skill in enumerate(unique_all_skills[:10], 1):
    print(f"   {i:2d}. {skill}")

Extracting skills from merged DataFrame...
SAP Technologies: 20260 unique skills
Programming Languages: 4005 unique skills
Databases: 1 unique skills
Web Technologies: 1 unique skills
Business Skills: 435 unique skills
Technical Skills: 1 unique skills
Development Tools: 1 unique skills
Communication Skills: 1 unique skills
Other Skills: 13513 unique skills

📊 Total skills before deduplication: 38,218
📊 Total unique skills after deduplication: 28,136
Development Tools: 1 unique skills
Communication Skills: 1 unique skills
Other Skills: 13513 unique skills

📊 Total skills before deduplication: 38,218
📊 Total unique skills after deduplication: 28,136

✅ Successfully saved all skills to: all_skills_combined.json
📋 Breakdown by category:
   - SAP_Technologies: 20,260 skills
   - Programming_Languages: 4,005 skills
   - Databases: 1 skills
   - Web_Technologies: 1 skills
   - Business_Skills: 435 skills
   - Technical_Skills: 1 skills
   - Development_Tools: 1 skills
   - Communication_Skil

In [35]:
# Select specific columns from merged_df - need square brackets for multiple columns
final_merged_df = merged_df[['SAP_Technologies', 'Programming_Languages', 'Other_Skills']]
print("Shape of final merged DataFrame:", final_merged_df.shape)
print("\nFirst 5 rows:")
print(final_merged_df.head())

# Also display some statistics
print(f"\nData availability:")
for col in final_merged_df.columns:
    non_empty = (final_merged_df[col] != '').sum()
    print(f"- {col}: {non_empty:,} records with data ({non_empty/len(final_merged_df)*100:.1f}%)")

Shape of final merged DataFrame: (9999, 3)

First 5 rows:
                                    SAP_Technologies Programming_Languages  \
0  Data Loss Prevention; Networking; Firewall; Se...                         
1  Litigation Management; Litigation; contracts; ...               vetting   
2       SFDC; Salesforce; Salesforce developer; Java                         
3   Intrusion Detection; Networking; LAN; Linux; WAN              Firewall   
4  Business Development Management; Concept Selli...              caterers   

              Other_Skills  
0   Network Administration  
1                 drafting  
2                     J2ee  
3  Unix; Technical Support  
4      horeca; Bakery; B2B  

Data availability:
- SAP_Technologies: 9,999 records with data (100.0%)
- Programming_Languages: 6,983 records with data (69.8%)
- Other_Skills: 7,716 records with data (77.2%)


In [36]:
import json
from datetime import datetime

# Extract all unique skills from the three main categories
print("Extracting all skills...")

# Extract SAP Technologies
sap_skills = final_merged_df['SAP_Technologies'].dropna().str.split('; ').explode().dropna().unique()
print(f"SAP Technologies: {len(sap_skills):,} unique skills")

# Extract Programming Languages  
prog_skills = final_merged_df['Programming_Languages'].dropna().str.split('; ').explode().dropna().unique()
print(f"Programming Languages: {len(prog_skills):,} unique skills")

# Extract Other Skills
other_skills = final_merged_df['Other_Skills'].dropna().str.split('; ').explode().dropna().unique()
print(f"Other Skills: {len(other_skills):,} unique skills")

# Combine all skills into a single array
all_skills_combined = []
all_skills_combined.extend(sap_skills.tolist())
all_skills_combined.extend(prog_skills.tolist())
all_skills_combined.extend(other_skills.tolist())

# Remove duplicates while preserving order
unique_skills = list(dict.fromkeys(all_skills_combined))

print(f"\nTotal skills before deduplication: {len(all_skills_combined):,}")
print(f"Total unique skills after deduplication: {len(unique_skills):,}")

# Create the JSON structure with all skills in a single array
skills_json_data = {
    "skills": unique_skills,
    "total_count": len(unique_skills),
    "category_breakdown": {
        "SAP_Technologies": len(sap_skills),
        "Programming_Languages": len(prog_skills), 
        "Other_Skills": len(other_skills)
    },
    "generated_on": datetime.now().isoformat(),
    "source": "merged_skill_data_10_columns.csv"
}

# Save to JSON file
output_json = 'all_skills_single_array.json'
with open(output_json, 'w', encoding='utf-8') as f:
    json.dump(skills_json_data, f, indent=2, ensure_ascii=False)

print(f"\n✅ Successfully saved all skills to: {output_json}")
print(f"📊 Total unique skills: {len(unique_skills):,}")
print(f"📋 Breakdown:")
for category, count in skills_json_data['category_breakdown'].items():
    print(f"   - {category}: {count:,} skills")

# Show first 10 skills as sample
print(f"\n📝 First 10 skills in the combined array:")
for i, skill in enumerate(unique_skills[:10], 1):
    print(f"   {i:2d}. {skill}")

Extracting all skills...
SAP Technologies: 20,260 unique skills
Programming Languages: 4,005 unique skills
Other Skills: 13,513 unique skills

Total skills before deduplication: 37,778
Total unique skills after deduplication: 28,083

✅ Successfully saved all skills to: all_skills_single_array.json
📊 Total unique skills: 28,083
📋 Breakdown:
   - SAP_Technologies: 20,260 skills
   - Programming_Languages: 4,005 skills
   - Other_Skills: 13,513 skills

📝 First 10 skills in the combined array:
    1. Data Loss Prevention
    2. Networking
    3. Firewall
    4. Security
    5. Litigation Management
    6. Litigation
    7. contracts
    8. Legal
    9. SFDC
   10. Salesforce
